In [1]:
#Importing the libraries
import numpy as np
import pandas as pd
import os
import io
import pdb
import sklearn
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn import linear_model
import pyswarms as ps
from statistics import mean, stdev
from sklearn.model_selection import cross_val_score
from tpot import TPOTClassifier
import matplotlib.pyplot as plt

C:\Users\ykond\anaconda3\lib\site-packages\tpot\builtins\__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


In [2]:
def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path=os.path.join(root, filename)
            lines=[]
            f=io.open(path, 'r', encoding='latin1')
            for line in f:
                lines.append(line)
            f.close
            message='\n'.join(lines)
            yield path, message

def dataFrameFromDirectory(path, classification):
    rows=[]
    index=[]
    for filename, message in readFiles(path):
        rows.append({'message':message, 'class':classification})
        index.append(filename)
    return pd.DataFrame(rows, index=index)

data=pd.DataFrame({'message':[], 'class':[]})

data=data.append(dataFrameFromDirectory('D:/Major Project/App_Data_Set/Spam_Assassin_Dataset/Spam','Spam'))
data=data.append(dataFrameFromDirectory('D:/Major Project/App_Data_Set/Spam_Assassin_Dataset/Ham','Ham'))
X=data['message']
y=data['class']

vectorizer=CountVectorizer(stop_words='english',lowercase=True)

'''Pre-processing'''
IDF = TfidfVectorizer().fit_transform(X)
Tr_tokens=vectorizer.fit_transform(X)

scores=np.array

In [3]:
#Applying Stratified K_fold_Cross_Validation
def SKF_Split(x):
    skf=StratifiedKFold(n_splits=x,shuffle=True)
    #Stratified K_Fold_spliting
    for train_index, test_index in skf.split(Tr_tokens,y):
        global X_train, X_test, y_train, y_test
        X_train, X_test = Tr_tokens[train_index], Tr_tokens[test_index]
        y_train, y_test = y[train_index], y[test_index]
    return X_train, X_test, y_train, y_test
SKF_Split(4)

(<16138x189022 sparse matrix of type '<class 'numpy.int64'>'
 	with 3922144 stored elements in Compressed Sparse Row format>,
 <5379x189022 sparse matrix of type '<class 'numpy.int64'>'
 	with 1328066 stored elements in Compressed Sparse Row format>,
 D:/Major Project/App_Data_Set/Spam_Assassin_Dataset/Spam\20021009_spam\spam\0000.7b1b73cf36cf9dbc3d64e3f2ee2b91f1    Spam
 D:/Major Project/App_Data_Set/Spam_Assassin_Dataset/Spam\20021009_spam\spam\0001.bfc8d64d12b325ff385cca8d07b84288    Spam
 D:/Major Project/App_Data_Set/Spam_Assassin_Dataset/Spam\20021009_spam\spam\0002.24b47bb3ce90708ae29d0aec1da08610    Spam
 D:/Major Project/App_Data_Set/Spam_Assassin_Dataset/Spam\20021009_spam\spam\0003.4b3d943b8df71af248d12f8b2e7a224a    Spam
 D:/Major Project/App_Data_Set/Spam_Assassin_Dataset/Spam\20021009_spam\spam\0005.1f42bb885de0ef7fc5cd09d34dc2ba54    Spam
                                                                                                                      ... 
 D:/Major P

In [4]:
def GA_MNB():
    tpot_config = {'sklearn.naive_bayes.MultinomialNB': {'alpha': [1e-3, 1e-2, 1e-1, 1., 10., 100.],'fit_prior': [True, False]}}

    tpot = TPOTClassifier(generations=10, population_size=40, verbosity=2,config_dict=tpot_config,offspring_size=20)
    tpot.fit(X_train, y_train)
    pred=tpot.predict(X_test)
    return metrics.accuracy_score(pred, y_test), metrics.precision_score(y_test,pred,pos_label='Spam'), metrics.recall_score(y_test,pred,pos_label='Spam'), (metrics.f1_score(y_test, pred, pos_label='Spam')*100)
GA_MNB()

Optimization Progress:   0%|          | 0/240 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.9946088398345896

Generation 2 - Current best internal CV score: 0.9946088398345896

Generation 3 - Current best internal CV score: 0.9946088398345896

Generation 4 - Current best internal CV score: 0.9946088398345896

Generation 5 - Current best internal CV score: 0.9946088398345896

Generation 6 - Current best internal CV score: 0.9946088398345896

Generation 7 - Current best internal CV score: 0.9946088398345896

Generation 8 - Current best internal CV score: 0.9946088398345896

Generation 9 - Current best internal CV score: 0.9946088398345896

Generation 10 - Current best internal CV score: 0.9946088398345896

Best pipeline: MultinomialNB(input_matrix, alpha=0.001, fit_prior=False)


(0.9957241122885294, 0.9948119325551232, 0.9903163331181407, 99.25590423811065)

In [5]:
def GA_DTC():
    tpot_config = {'sklearn.tree.DecisionTreeClassifier': {'criterion': ["gini", "entropy"],'max_depth': range(1, 11),'min_samples_split': range(2, 21),'min_samples_leaf': range(1, 21)}}

    tpot = TPOTClassifier(generations=10, population_size=40, verbosity=2,config_dict=tpot_config,offspring_size=20)
    tpot.fit(X_train, y_train)
    pred=tpot.predict(X_test)
    return metrics.accuracy_score(pred, y_test), metrics.precision_score(y_test,pred,pos_label='Spam'), metrics.recall_score(y_test,pred,pos_label='Spam'), (metrics.f1_score(y_test, pred, pos_label='Spam')*100)
GA_DTC()

Optimization Progress:   0%|          | 0/240 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.9751510355047195

Generation 2 - Current best internal CV score: 0.9751510355047195

Generation 3 - Current best internal CV score: 0.9751510355047195

Generation 4 - Current best internal CV score: 0.9827112586682457

Generation 5 - Current best internal CV score: 0.9827112586682457

Generation 6 - Current best internal CV score: 0.9827112586682457

Generation 7 - Current best internal CV score: 0.9827112586682457

Generation 8 - Current best internal CV score: 0.9827112586682457

Generation 9 - Current best internal CV score: 0.9827112586682457

Generation 10 - Current best internal CV score: 0.9827112586682457

Best pipeline: DecisionTreeClassifier(input_matrix, criterion=gini, max_depth=10, min_samples_leaf=1, min_samples_split=12)


(0.9830823573154861, 0.9758485639686684, 0.9651387992253067, 97.0464135021097)

In [6]:
def GA_SGDC():
    tpot_config = {'sklearn.linear_model.SGDClassifier': {'loss': ['log', 'hinge', 'modified_huber', 'squared_hinge', 'perceptron'],'penalty': ['elasticnet'],'alpha': [0.0, 0.01, 0.001],
                                                          'learning_rate': ['invscaling', 'constant'],'fit_intercept': [True, False],'l1_ratio': [0.25, 0.0, 1.0, 0.75, 0.5],'eta0': [0.1, 1.0, 0.01],
                                                          'power_t': [0.5, 0.0, 1.0, 0.1, 100.0, 10.0, 50.0]}}

    tpot = TPOTClassifier(generations=10, population_size=40, verbosity=2,config_dict=tpot_config,offspring_size=20)
    tpot.fit(X_train, y_train)
    pred=tpot.predict(X_test)
    return metrics.accuracy_score(pred, y_test), metrics.precision_score(y_test,pred,pos_label='Spam'), metrics.recall_score(y_test,pred,pos_label='Spam'), (metrics.f1_score(y_test, pred, pos_label='Spam')*100)
GA_SGDC()

Optimization Progress:   0%|          | 0/240 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.9972733737835465

Generation 2 - Current best internal CV score: 0.9972733737835465

Generation 3 - Current best internal CV score: 0.9972733737835465

Generation 4 - Current best internal CV score: 0.9972733737835465

Generation 5 - Current best internal CV score: 0.9972733737835465

Generation 6 - Current best internal CV score: 0.9972733737835465

Generation 7 - Current best internal CV score: 0.9972733737835465

Generation 8 - Current best internal CV score: 0.9972733737835465

Generation 9 - Current best internal CV score: 0.9973973087206804

Generation 10 - Current best internal CV score: 0.9973973087206804

Best pipeline: SGDClassifier(CombineDFs(input_matrix, input_matrix), alpha=0.0, eta0=0.1, fit_intercept=False, l1_ratio=0.0, learning_rate=constant, loss=log, penalty=elasticnet, power_t=10.0)


(0.9960959286112661, 0.9973958333333334, 0.9890251775338929, 99.31928687196111)

In [4]:
def GA_RFC():
    tpot_config = {'sklearn.ensemble.RandomForestClassifier': {'n_estimators': [100],'criterion': ["gini", "entropy"],'max_features': np.arange(0.05, 1.01, 0.05),
                                                               'min_samples_split': range(2, 21),'min_samples_leaf':  range(1, 21),'bootstrap': [True, False]}}

    tpot = TPOTClassifier(generations=10, population_size=40, verbosity=2,config_dict=tpot_config,offspring_size=20)
    tpot.fit(X_train, y_train)
    pred=tpot.predict(X_test)
    return metrics.accuracy_score(pred, y_test), metrics.precision_score(y_test,pred,pos_label='Spam'), metrics.recall_score(y_test,pred,pos_label='Spam'), (metrics.f1_score(y_test, pred, pos_label='Spam')*100)
GA_RFC()

2022-07-21 09:55:43,866 - stopit - WARNING - Code block execution exceeded 10 seconds timeout
Traceback (most recent call last):
  File "C:\Users\ykond\anaconda3\lib\site-packages\stopit\utils.py", line 145, in wrapper
    result = func(*args, **kwargs)
  File "C:\Users\ykond\anaconda3\lib\site-packages\tpot\decorators.py", line 57, in time_limited_call
    func(*args)
  File "C:\Users\ykond\anaconda3\lib\site-packages\sklearn\pipeline.py", line 382, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\ykond\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 476, in fit
    trees = Parallel(
  File "C:\Users\ykond\anaconda3\lib\site-packages\joblib\parallel.py", line 1046, in __call__
    while self.dispatch_one_batch(iterator):
  File "C:\Users\ykond\anaconda3\lib\site-packages\joblib\parallel.py", line 861, in dispatch_one_batch
    self._dispatch(tasks)
  File "C:\Users\ykond\anaconda3\lib\site-packages\joblib\parallel.py", line 779, in _d

Optimization Progress:   0%|          | 0/240 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: -inf


RuntimeError: There was an error in the TPOT optimization process. This could be because the data was not formatted properly, or because data for a regression problem was provided to the TPOTClassifier object. Please make sure you passed the data to TPOT correctly. If you enabled PyTorch estimators, please check the data requirements in the online documentation: https://epistasislab.github.io/tpot/using/

In [ ]:
def GA_MLPC():
    tpot_config = {'sklearn.neural_network.MLPClassifier': {'alpha': [1e-4, 1e-3, 1e-2, 1e-1],'learning_rate_init': [1e-3, 1e-2, 1e-1, 0.5, 1.]}}

    tpot = TPOTClassifier(generations=10, population_size=40, verbosity=2,config_dict=tpot_config,offspring_size=20)
    tpot.fit(X_train, y_train)
    pred=tpot.predict(X_test)
    return metrics.accuracy_score(pred, y_test), metrics.precision_score(y_test,pred,pos_label='Spam'), metrics.recall_score(y_test,pred,pos_label='Spam'), (metrics.f1_score(y_test, pred, pos_label='Spam')*100)
GA_MLPC()

2022-07-25 11:56:24,254 - stopit - WARNING - Code block execution exceeded 10 seconds timeout
Traceback (most recent call last):
  File "C:\Users\ykond\anaconda3\lib\site-packages\stopit\utils.py", line 145, in wrapper
    result = func(*args, **kwargs)
  File "C:\Users\ykond\anaconda3\lib\site-packages\tpot\decorators.py", line 57, in time_limited_call
    func(*args)
  File "C:\Users\ykond\anaconda3\lib\site-packages\sklearn\pipeline.py", line 378, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "C:\Users\ykond\anaconda3\lib\site-packages\sklearn\pipeline.py", line 336, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "C:\Users\ykond\anaconda3\lib\site-packages\joblib\memory.py", line 349, in __call__
    return self.func(*args, **kwargs)
  File "C:\Users\ykond\anaconda3\lib\site-packages\sklearn\pipeline.py", line 870, in _fit_transform_one
    res = transformer.fit_transform(X, y, **fit_params)
  File "C:\Users\ykond\anaconda3\lib\site-packages\sklear

In [ ]:
def GA_GBC():
    tpot_config = {'sklearn.ensemble.GradientBoostingClassifier': {
        'n_estimators': [100],
        'learning_rate': [1e-3, 1e-2, 1e-1, 0.5, 1.],
        'max_depth': range(1, 11),
        'min_samples_split': range(2, 21),
        'min_samples_leaf': range(1, 21),
        'subsample': np.arange(0.05, 1.01, 0.05),
        'max_features': np.arange(0.05, 1.01, 0.05)}}

    tpot = TPOTClassifier(generations=10, population_size=40, verbosity=2, config_dict=tpot_config, offspring_size=20)
    tpot.fit(X_train, y_train)
    pred=tpot.predict(X_test)
    return metrics.accuracy_score(pred, y_test), metrics.precision_score(y_test,pred,pos_label='Spam'), metrics.recall_score(y_test,pred,pos_label='Spam'), (metrics.f1_score(y_test, pred, pos_label='Spam')*100)
GA_GBC()

2022-07-25 12:02:55,353 - stopit - WARNING - Code block execution exceeded 10 seconds timeout
Traceback (most recent call last):
  File "C:\Users\ykond\anaconda3\lib\site-packages\stopit\utils.py", line 145, in wrapper
    result = func(*args, **kwargs)
  File "C:\Users\ykond\anaconda3\lib\site-packages\tpot\decorators.py", line 57, in time_limited_call
    func(*args)
  File "C:\Users\ykond\anaconda3\lib\site-packages\sklearn\pipeline.py", line 382, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\ykond\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py", line 668, in fit
    n_stages = self._fit_stages(
  File "C:\Users\ykond\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py", line 745, in _fit_stages
    raw_predictions = self._fit_stage(
  File "C:\Users\ykond\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py", line 247, in _fit_stage
    tree.fit(X, residual, sample_weight=sample_weight, check_input=False)
  File "C:\Users\ykond\anaconda3\l

Optimization Progress:   0%|          | 0/240 [00:00<?, ?pipeline/s]

2022-07-25 15:12:29,946 - stopit - WARNING - Code block execution exceeded 10 seconds timeout
Traceback (most recent call last):
  File "C:\Users\ykond\anaconda3\lib\site-packages\stopit\utils.py", line 145, in wrapper
    result = func(*args, **kwargs)
  File "C:\Users\ykond\anaconda3\lib\site-packages\tpot\decorators.py", line 57, in time_limited_call
    func(*args)
  File "C:\Users\ykond\anaconda3\lib\site-packages\sklearn\pipeline.py", line 382, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\ykond\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py", line 668, in fit
    n_stages = self._fit_stages(
  File "C:\Users\ykond\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py", line 745, in _fit_stages
    raw_predictions = self._fit_stage(
  File "C:\Users\ykond\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py", line 250, in _fit_stage
    loss.update_terminal_regions(
  File "C:\Users\ykond\anaconda3\lib\site-packages\sklearn\ensemble\_gb_lo


Generation 1 - Current best internal CV score: 0.9951044067846073


2022-07-25 16:47:12,926 - stopit - WARNING - Code block execution exceeded 10 seconds timeout
Traceback (most recent call last):
  File "C:\Users\ykond\anaconda3\lib\site-packages\stopit\utils.py", line 145, in wrapper
    result = func(*args, **kwargs)
  File "C:\Users\ykond\anaconda3\lib\site-packages\tpot\decorators.py", line 57, in time_limited_call
    func(*args)
  File "C:\Users\ykond\anaconda3\lib\site-packages\sklearn\pipeline.py", line 382, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\ykond\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py", line 668, in fit
    n_stages = self._fit_stages(
  File "C:\Users\ykond\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py", line 745, in _fit_stages
    raw_predictions = self._fit_stage(
  File "C:\Users\ykond\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py", line 247, in _fit_stage
    tree.fit(X, residual, sample_weight=sample_weight, check_input=False)
  File "C:\Users\ykond\anaconda3\l

In [ ]:
def GA_ABC():
    tpot_config = {'sklearn.ensemble.AdaBoostClassifier': {
        'n_estimators': [100],
        'learning_rate': [1e-3, 1e-2, 1e-1, 0.5, 1.0]}}
    
    tpot = TPOTClassifier(generations=10, population_size=40, verbosity=2, config_dict=tpot_config, offspring_size=20)
    tpot.fit(X_train, y_train)
    pred=tpot.predict(X_test)
    return metrics.accuracy_score(pred, y_test), metrics.precision_score(y_test,pred,pos_label='Spam'), metrics.recall_score(y_test,pred,pos_label='Spam'), (metrics.f1_score(y_test, pred, pos_label='Spam')*100)
GA_ABC()

In [ ]:
def Main_GA():
    SKF_Split(4)
    acc1=GA_MNB()
    acc2=GA_SGDC()
    acc3=GA_DTC()
    acc4=GA_RFC()
    acc5=GA_MLPC()
    acc6=GA_GBC()
    acc7=GA_ABC()
    
    
    Accuracy_Table=[('Stochastic Gradient Descent',acc2[0],acc2[1],acc2[2],acc2[3]),
            ('Multinomial Naive BAYES',acc1[0],acc1[1],acc1[2],acc1[3]),
            ('Random Forest',acc4[0],acc4[1],acc4[2],acc4[3]),
            ('Decision Tree',acc3[0],acc3[1],acc3[2],acc3[3]),
            ('Multi-later Perceptron',acc5[0],acc5[1],acc5[2],acc5[3]),
            ('Gradient Boost Classifier',acc6[0],acc6[1],acc6[2],acc6[3]),
            ('Adaboost Classifier',acc7[0],acc7[1],acc7[2],acc7[3])
           ]
    Result_table=pd.DataFrame(Accuracy_Table,columns=["Classifier","Testing Accuracy","Precision Score","Recall Score","F1-Score"])
    
    N = 4
    ind = np.arange(N) 
    width = 0.08

    MNB_vals = [acc1[0],acc1[1],acc1[2],acc1[3]]
    bar1 = plt.bar(ind, MNB_vals, width, color = 'r')

    SGDC_vals = [acc2[0],acc2[1],acc2[2],acc2[3]]
    bar2 = plt.bar(ind+width, SGDC_vals, width, color='g')

    DTC_vals = [acc3[0],acc3[1],acc3[2],acc3[3]]
    bar3 = plt.bar(ind+width*2, DTC_vals, width, color = 'b')

    RFC_vals = [acc4[0],acc4[1],acc4[2],acc4[3]]
    bar4 = plt.bar(ind+width*3, RFC_vals, width, color='y')
    
    MLPC_vals = [acc5[0],acc5[1],acc5[2],acc5[3]]
    bar5 = plt.bar(ind+width*4, MLPC_vals, width, color='c')
    
    GBC_vals = [acc6[0],acc6[1],acc6[2],acc6[3]]
    bar6 = plt.bar(ind+width*5, GBC_vals, width, color='m')
    
    ABC_vals = [acc7[0],acc7[1],acc7[2],acc7[3]]
    bar7 = plt.bar(ind+width*6, ABC_vals, width, color='k')
    
        
    plt.xlabel("Accuracy Fields")
    plt.ylabel("Accuracies")
    plt.title("Accuracies for 75-25 Split")

    plt.xticks(ind+width,['Testing Accuracy', 'Precision Score', 'Recall Score', 'F1-Score'])
    plt.legend( (bar1, bar2, bar3, bar4, bar5, bar6, bar7), ('MNB', 'SGDC', 'DTC', 'RFC', 'MLPC', 'GBC', 'ABC') )
    plt.show()
    
    return Result_table
Main_GA()
